###### 贪婪与非贪婪 

In [ ]:
m = re.match("([abc])", "abc")
print(m.group(1)) # 这里组1的匹配是非贪婪的
m = re.search("([abc])+", "abc")
print(m.group(1)) # 这里组1的匹配是贪婪的，因此匹配到c

###### 2.2 在字符串的开头或结尾进行文本匹配 
    str.startswith()
    str.endswith()

In [5]:
# str.endswith(str/tuple)
# 当参数只有一个元素时为字符串，若有多个元素，则至于元组内。
file = ('name.py', 'te.py', 'www.ook.com', 'www.asd.cn')
L = list(n for n in file if n.endswith('.py'))
print(L)

r = 'ppwqheg.com'.endswith('.com')
print(r)

['name.py', 'te.py']
True


###### 2.5 查找和替换文本
    re.sub(pattern, replacement, string)
    正则表达式捕获分组，及捕获组的后向引用

In [17]:
import re
p = re.compile(r'(\d+)/(\d)+/(\d+)')
date = 'Today is 11/29/2019, PyCon starts 3/13/2013.'
result = p.sub(r'\3-\1-\2', date) # \3为对3号捕获组的引用，\2, \1同理
print(result)

# 对于更复杂的情况， replacement可以为函数
from calendar import month_abbr

def process(m):
    month_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), month_name, m.group(3))

result = p.sub(process, date)
print(result)

Today is 2019-11-9, PyCon starts 2013-3-3.
Today is 9 Nov 2019, PyCon starts 3 Mar 2013.


In [73]:
# 关于正则表达式中原始字符串的思考
# 以r'\\'为例，使用原始字符串后，'\'失去转义作用。此时将匹配python代码'\\'；
# 但是由于在python代码中'\'代表转义，因此在计算机的认知中，'\\'仅为字面值，打印值为'\'.

import re
p = re.compile(r'/\*((?:.|\n)*?)\*/')
r = p.findall('''/* this is a
              multiline comment */''')
print(str(r))
r = re.sub(r'(?:\s|\\n)+', ' ', str(r))
print(r)
print('asd\nlkj')

[' this is a\n              multiline comment ']
[' this is a multiline comment ']
asd
lkj


In [2]:
key = ['a', 'x', 'c', 'd']
value = [1, 2, 3, 4]
def iter_v(values):
    for v in values:
        yield v
d = dict.fromkeys(key, iter_v(value))
print(d)

{'a': <generator object iter_v at 0x000001D8793EBC00>, 'x': <generator object iter_v at 0x000001D8793EBC00>, 'c': <generator object iter_v at 0x000001D8793EBC00>, 'd': <generator object iter_v at 0x000001D8793EBC00>}


###### 2.15 给字符串中的变量名做插值处理
    str.format()
    str.format_map() 结合 vars()

In [35]:
# vars(x)函数返回x的__dict__属性。
# 关于dict属性，（个人便于理解）可认为是赋值关系，即存在键值对。类、类实例有各自的__dict__属性。
# 若vars()的参数为空，则返回

class Dict:
    a = 1
    b = 2
    
    def __init__(self):
        self.c = 3
        self.d = 4
        
d = Dict()
print(Dict.__dict__)
print(d.__dict__)

#========分割线==========
print('='*15, end='\n\n')


# str.format_map()相当于 str.format(**mapping)
# 下例即相当于 s.format(c=3, d=4)
s = 'c equals {c}.'
print(s.format_map(vars(d)))

{'__module__': '__main__', 'a': 1, 'b': 2, '__init__': <function Dict.__init__ at 0x000001D8794AC2F0>, '__dict__': <attribute '__dict__' of 'Dict' objects>, '__weakref__': <attribute '__weakref__' of 'Dict' objects>, '__doc__': None}
{'c': 3, 'd': 4}

c equals 3.


In [42]:
import textwrap
s = '''
       sdh
       second
       third
    '''
print(len(s))
print(textwrap.dedent(s))

43

sdh
second
third



###### 2.18 文本分词
    namedtuple()

In [79]:
from collections import namedtuple
import re, pprint

# 将下列字符串进行分词处理
text = 'foo = 3'

# 编译正则表达式，用于匹配字母，空白符，等号，数字
Str = r'(?P<Str>[a-zA-Z]+)'
Whi = r'(?P<Whi>\s+)'
Equ = r'(?P<Equ>\=)'
Num = r'(?P<Num>\d+)'

p = re.compile('|'.join([Str, Whi, Equ, Num]))

Store = namedtuple('Store', ['type', 'value'])

def iter_str(text):
    for match in p.finditer(text):
        print(match.group())
        yield Store(match.lastgroup, match.group())
        
l = list(iter_str(text))
pprint.pprint(l)

foo
 
=
 
3
[Store(type='Str', value='foo'),
 Store(type='Whi', value=' '),
 Store(type='Equ', value='='),
 Store(type='Whi', value=' '),
 Store(type='Num', value='3')]


In [105]:
# 拓展   正则re,match,lastindex属性解析
    # re.match.lastindex属性返回的是最后一个完成匹配的捕获组的编号。
    # 捕获组的编号按从左往右，由外到内的顺序编订。简单来说就是从左往右，‘（’出现的顺序就是编号的顺序。
    # 捕获组的匹配则按从左往右，由内到外的顺序匹配。这就导致了捕获组的匹配顺序与编号不一致。
    # 具体例子如下：
    
import re
s = 'ab'

print('测试1：')
r = re.match(r'(?P<ab>(?P<a>a)(?P<b>b))', s)# 匹配顺序为{组名a（编号2）} - {组名b(编号3)} - {组名ab(编号1)}
print(r.lastindex)                          # 按匹配顺序，最外层的捕获组ab为最后匹配，故返回组号1
print(r.lastgroup)
print(r.groupdict())
r = re.match(r'(?P<a>a)(?P<b>b)', s)
print(r.lastindex)
print(r.lastgroup)
print(r.groupdict())

print('测试2：')
r = re.match(r'(?P<A>(?P<a>a))(?P<b>b)', s)
print(r.lastindex)
print(r.lastgroup)
print(r.groupdict())
r = re.match(r'(?P<a>a)(?P<B>(?P<b>b))', s)
print(r.lastindex)
print(r.lastgroup)
print(r.groupdict())

print('测试3：')
r = re.match(r'(?P<A>(?P<a>a))', s)
print(r.lastindex)
print(r.lastgroup)
print(r.groupdict())

测试1：
1
ab
{'ab': 'ab', 'a': 'a', 'b': 'b'}
2
b
{'a': 'a', 'b': 'b'}
测试2：
3
b
{'A': 'a', 'a': 'a', 'b': 'b'}
2
B
{'a': 'a', 'B': 'b', 'b': 'b'}
测试3：
1
A
{'A': 'a', 'a': 'a'}


In [111]:
from collections import namedtuple
n = namedtuple('tu', ['x', 'y'])
p = n(1,2)
p

tu(x=1, y=2)

In [14]:
round(float(3.34), 1)

3.3

###### 2.19 编写一个简单的递归下降解析器

In [43]:
from collections import namedtuple
import re

# 将 数字、加、减、乘、除、左括号、右括号、空格 加工成文本分词

NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
MINUS = r'(?P<MINUS>-)' 
TIMES = r'(?P<TIMES>\*)'
DIVIDE = r'(?P<DIVIDE>/)'
LPAREN = r'(?P<LPAREN>\()'
RPAREN = r'(?P<RPAREN>\))'
WS = r'(?P<WS>\s+)'

p = re.compile('|'.join([NUM, PLUS, MINUS, TIMES, DIVIDE, LPAREN, RPAREN, WS]))

Stoken = namedtuple('Stoken', ['type', 'value'])

def generator(text):
    for match in p.finditer(text):
        stoken = Stoken(match.lastgroup, match.group())
        if stoken.type != 'WS':
            yield stoken


class Count:
    '''
    定义一个计算器类，用于进行可包含括号的整数四则运算；
    算式中的小数会被四舍五入后进行运算；
    最终结果保留两位小数。
    '''
    
    def parse(self, text):
        self.LPAREN = 0             # 记录左括号的读取数量，防止因左右括号数量不匹配而计算错误
        self.pops = generator(text) # 创建迭代器，用于读取算式
        self.c_pop = None           # 参与运算的值
        self.n_pop = None          #  取出的值，尚未参与运算
        self._get_next()           # 取出第一个值
        r = round(self.express(), 2)
        return r
    
    def _get_next(self):
        # 让取出的值参与运算，并取出下一个值备用
        self.c_pop, self.n_pop = self.n_pop, next(self.pops, None)
    
    def _accept(self, v_type):
        # 该函数用于判断已取出的值是否为v_type类型，结果以布尔值的形式返回
        if self.n_pop and self.n_pop.type == v_type:
            self._get_next()  
            return True
        else:
            return False
        
    def _check_syntax(self):
        '''该函数暂未使用'''
        # 防止出现右括号后面直接连接括号而得出错误结果
        check_l = ['LPAREN', 'RPAREN']
        if self.n_pop.type == 'LPAREN':
            raise SyntaxError('括号使用错误')
        elif self.LPAREN == 0 and self.n_pop.type == 'RPAREN':
            raise SyntaxError('括号使用错误')
        
    def _expect(self, v_type):
        if self.n_pop and self.n_pop.type == v_type:
            self.LPAREN -= 1
            self._get_next()
        else:
            raise SyntaxError('输入有误')
    
    def express(self):
        # 进行一级加减运算， 即（运算得到的常数） +/- （运算得到的常数）
        
        value = self.term() # 获取常数的值
        while self._accept('PLUS') or self._accept('MINUS'):
            lable = self.c_pop
            right = self.term()
            if lable.type == 'PLUS':
                value += right
            else:
                value -= right
        return value
    
    def term(self):
        # 进行二级乘除运算， 即（运算得到的因数） *\/ （运算得到的因数）
        
        value = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            lable = self.c_pop
            right = self.factor()
            if lable.type == 'TIMES':
                value *= right
            else:
                value /= right
        return value
    
    def factor(self):
        # 对获取的值进行判断，运算
        if self._accept('NUM'):
            return int(self.c_pop.value)
        elif self._accept('LPAREN'):
            self.LPAREN += 1
            value = self.express()
            self._expect('RPAREN')
            self._check_syntax()
            return value
        else:
            raise SyntaxError('输入有误')
couter = Count()
couter.parse('1 + (6 - 2) + (4 * 5) / 6')


8.33